In [2]:
import math
import numpy as np
import torch
import pkbar
import sys
from tqdm import tqdm
from torchsummary import summary
from torch.nn.functional import one_hot
from unet3d.config import *
from unet3d.tester import Tester
from unet3d.dice import dice_coef_torch_multiclass
from unet3d.transforms import *
from unet3d.unet3d_vgg16 import UNet3D_VGG16
from unet3d.transforms import val_transform
from utils.Visualization import ImageSliceViewer3D
import nrrd
import os 

%load_ext autoreload
%autoreload 2
%matplotlib inline

torch.manual_seed(0)
torch.backends.cudnn.benchmark = True # Speeds up stuff
torch.backends.cudnn.enabled = True
device = torch.device('cuda')

model_path = 'checkpoints/saiad1and18tervasc_27sep_epoch249.pth'
model_path = os.path.realpath(model_path)
print(model_path)

/Work/Users/acharneca/Vessel-Segmentation-pytorch/checkpoints/saiad1and18tervasc_27sep_epoch249.pth


In [2]:
## Test (assuming we have the truth segmentations)###
tester = Tester(
    model_path, 
    test_patient = 'SAIAD 13'
    )

# Predict from uniform spacing scan
temp = tester.predict(with_transforms = True, verbose=1) ### remove temp

## Post Processing ##
tester.apply_gaussian_blur(sigma=2, truncate=2);tester.clean_connected_components()


# Save uniform spacing prediction to folder
tester.save_uniform_spacing_segm()

# Resample to the original spacing of the scan and save
tester.resample_to_original_and_save()

dice_vals = tester.compute_dice()
print("Dice vals: ", dice_vals)



Patchifying scan...
Predicting on patches...


100%|████████████████████████████████████████████████████████████████████████████| 10/10 [01:04<00:00,  6.42s/it]


Unpatchifying...
Prediction done.
Dice vals:  tensor([0.9841, 0.9390, 0.0940, 0.2647, 0.7874])


In [3]:
tester.show_scan_vs_pred()

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [4]:
vol = temp['single_patch_predictions'].cpu().detach().numpy()[0,2]#[5,5,1,:,:,:,0]
input_ = temp['patch_input'].cpu().detach().numpy()[0,0]
print(vol.shape)
print(input_.shape)
ImageSliceViewer3D(input_, vol)

(96, 96, 96)
(96, 96, 96)


interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…

In [3]:
## Inference (assuming no truth segm is available) ##
tester = Tester(
    model_path, 
    test_patient = 'SAIAD 5 test'
    )

# Predict from uniform spacing scan
temp = tester.predict(with_transforms = True, verbose=1) ### remove temp

## Post Processing ##
tester.apply_gaussian_blur(sigma=2, truncate=2);tester.clean_connected_components()

# Save uniform spacing prediction to folder
tester.save_uniform_spacing_segm()

# Resample to the original spacing of the scan and save
tester.resample_to_original_and_save()


No truth segmentation found: Inference mode enabled
Patchifying scan...
Predicting on patches...


100%|████████████████████████████████████████████████████████████████████████████| 10/10 [01:29<00:00,  8.94s/it]


Unpatchifying...
Prediction done.


In [4]:
tester.show_scan_vs_pred()

interactive(children=(RadioButtons(description='Slice plane selection:', options=('x-y', 'y-z', 'z-x'), style=…